In [97]:

def create_equity_curve(df,z0,z1,pos):
    ec=df.diff(axis=0)
    ec.iloc[0]=0
    
    i=1
    while i<len(ec):
        ec.iloc[i]=ec.iloc[i-1]+ec.iloc[i]*z0*z1*pos
        i+=1
    return ec
    
    

def pos():
    pos=1
    while True:
        a=raw_input("Please input 'y' if position is long and 'n' if position is short: ").lower()
    
        if a=='y':
            break
        if a=='n':
            pos=pos*(-1)
            break
        else:
            print "Please input either 'y' or 'n' for position"
    return pos

def z_factor(product1,product2):
    z=1
    factor=1000
    if product1=='RB' and product2 =='RB':
        z=42
    elif product1=='HO' and product2 =='HO':
        z=42
    elif product1=='RB' and product2 =='HO':
        z=42
    elif product1=='HO' and product2 =='RB':
        z=42
    elif product1=='G' and product2 =='G':
        factor=100
    elif product1=='G' and product2 =='B':
        z=(1/7.45)
    
    else:
        z=1
        factor=1000
    
    return z,factor
        
def create_sprd(dataframe1, dataframe2):
   
    dataframe1.reset_index(inplace=True)
    dataframe2.reset_index(inplace=True)
    date=dataframe1['Date']
    
    dataframe1.drop(columns=['Date'],inplace=True)
    dataframe2.drop(columns=['Date'],inplace=True)
    
    while True:
        sprd_type=int(raw_input('Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: '))
        
        if sprd_type==1:
            
            new_df=dataframe1*42-dataframe2
            new_df['Date']=date
            new_df.set_index('Date',inplace=True)
            new_df.fillna(method='bfill',inplace=True)
            new_df.fillna(method='ffill',inplace=True)
            
            return new_df
            break
        
        elif sprd_type==2:
            
            new_df= dataframe1-dataframe2
            new_df['Date']=date
            new_df.set_index('Date',inplace=True)
            new_df.fillna(method='bfill',inplace=True)
            new_df.fillna(method='ffill',inplace=True)
            
            return new_df
            break
        
        else:
            print 'You have inputted incorrectly. Please try again and enter either integer 1 or 2.'
    

class Mth_1(object):
    
    def __init__(self):
        pass
        
    def pdt_type1(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no1(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self,start_mth,start_day,end_mth,end_day,i,j):

        count=0
        path = os.getcwd()
        
        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string

            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            #Join a date dataframe and the temp dataframe together
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date] # filter for start date

            temp=temp[temp['Date']<=end_date] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only

    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #don't need this, i believe
        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change', 'Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi
            
            

class Mth_2(object):
    
    def __init__(self):
        pass
        
    def pdt_type2(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no2(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self, start_mth, start_day, end_mth,end_day,i,j):

        count=0
        path = os.getcwd()

        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string
     
            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            #Join a date dataframe and the temp dataframe together
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date] # filter for start date

            temp=temp[temp['Date']<=end_date] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only
    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change','Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi
    
    
import numpy as np
import pandas as pd
import os

#from plotly import __version__
#import cufflinks as cf
#from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
#init_notebook_mode(connected=True)
#cf.go_offline()

%matplotlib inline

#user specify instrument and contract month

product1=''
month1=''
product2=''
month2=''

front_mth=Mth_1()
product1 = front_mth.pdt_type1()
month1=front_mth.mth_no1()

back_mth=Mth_2()
product2=back_mth.pdt_type2()
month2=back_mth.mth_no2()

        

Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 4
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 3
Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 4
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 4


In [98]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [99]:
cd $product1

C:\Users\boonsin.teo\Documents\Quandl\RB


In [100]:
cd $month1

C:\Users\boonsin.teo\Documents\Quandl\RB\Mar


In [101]:
start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')
i=int(raw_input('Please input 1 if start mth is the same as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')
            
j=int(raw_input('Please input 1 if end mth is the same as contract year and 0 if not: '))


front_month=front_mth.dataframe_create(start_mth,start_day,end_mth,end_day,i,j)

Please input start mth of seasonal period in MM format: 01
Please input start day of seasonal period in DD format: 09
Please input 1 if start mth is the same as contract year and 0 if not: 1
Please input end mth of seasonal period in MM format: 02
Please input end day of seasonal period in DD format: 10
Please input 1 if end mth is the same as contract year and 0 if not: 1


In [102]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\RB


In [103]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [104]:
cd ..

C:\Users\boonsin.teo\Documents


In [105]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [106]:
cd $product2

C:\Users\boonsin.teo\Documents\Quandl\RB


In [107]:
cd $month2

C:\Users\boonsin.teo\Documents\Quandl\RB\Apr


In [108]:
back_month=back_mth.dataframe_create(start_mth, start_day,end_mth,end_day,i,j)

In [109]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\RB


In [110]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [111]:
cd ..

C:\Users\boonsin.teo\Documents


In [112]:
front_month.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Date,,,,,,,,,,,,,,,,,,,,,
01/09,0.5697,0.7063,0.5310,NaN,NaN,0.8121,0.5940,0.8943,1.0206,NaN,...,1.1577,NaN,NaN,2.7689,2.7898,2.6552,1.3629,NaN,1.5972,1.8530
01/10,0.5672,0.7056,NaN,NaN,0.6721,0.8613,0.6014,0.8732,NaN,1.2274,...,NaN,NaN,2.4607,2.7792,2.8026,2.6797,NaN,NaN,1.5736,1.8518
01/11,0.5389,NaN,NaN,0.4023,0.6890,0.8582,0.5876,NaN,NaN,1.2298,...,NaN,2.1547,2.4901,2.7703,2.7543,NaN,NaN,1.1369,1.6197,1.8541
01/12,0.5256,NaN,0.5238,0.3917,0.7151,0.8686,NaN,NaN,1.0182,1.2393,...,1.1296,2.1106,2.4772,2.7370,NaN,NaN,1.3097,1.1097,1.6371,1.8656
01/13,NaN,0.6861,0.5193,0.3760,0.7167,NaN,NaN,0.8986,1.0001,1.2764,...,1.1894,2.0754,2.4615,2.7397,NaN,2.6470,1.3032,1.0821,1.6373,NaN


In [113]:
back_month.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Date,,,,,,,,,,,,,,,,,,,,,
01/09,0.5847,0.7213,0.5540,NaN,NaN,0.8446,0.6570,0.9373,1.0600,NaN,...,1.2952,NaN,NaN,2.9007,2.9386,2.8330,1.5999,NaN,1.7839,2.0351
01/10,0.5832,0.7220,NaN,NaN,0.7048,0.8933,0.6654,0.9182,NaN,1.2894,...,NaN,NaN,2.5543,2.9042,2.9546,2.8547,NaN,NaN,1.7604,2.0360
01/11,0.5610,NaN,NaN,0.4301,0.7195,0.8870,0.6521,NaN,NaN,1.2943,...,NaN,2.2544,2.5888,2.8954,2.9165,NaN,NaN,1.3530,1.8039,2.0333
01/12,0.5494,NaN,0.5476,0.4205,0.7431,0.8941,NaN,NaN,1.0597,1.3065,...,1.2666,2.2102,2.5816,2.8643,NaN,NaN,1.5498,1.3272,1.8215,2.0463
01/13,NaN,0.7055,0.5445,0.4065,0.7445,NaN,NaN,0.9416,1.0445,1.3429,...,1.3214,2.1772,2.5661,2.8659,NaN,2.8267,1.5410,1.3037,1.8199,NaN


In [114]:
Spread_Combi=create_sprd(front_month, back_month)

Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: 2


In [115]:
Spread_Combi.dropna(axis=0,how='all',inplace=True) # drop rows with all NAs

In [116]:
Spread_Combi.dropna(axis=1,how='all',inplace=True)# drop columns with all NAs

In [117]:
position=pos()
print 'PnL will be calculated basis 1kb lot size for standardization purposes.'

Please input 'y' if position is long and 'n' if position is short: n
PnL will be calculated basis 1kb lot size for standardization purposes.


In [118]:
zf=z_factor(product1, product2)

In [119]:
#PnL Series
pnl_series=(Spread_Combi.iloc[-1]-Spread_Combi.iloc[0])*zf[0]*zf[1]*position

In [120]:
#Win %
Win_Percent=float(pnl_series[pnl_series>0].count())/float(pnl_series.count())*100

In [121]:
#Largest Drawdown - use max and min
max_series=Spread_Combi.max()
min_series=Spread_Combi.min()
if position==1:
    max_drawdown=(min_series-Spread_Combi.iloc[0])*position*zf[0]*zf[1]
else:
    max_drawdown=(max_series-Spread_Combi.iloc[0])*position*zf[0]*zf[1]

In [122]:
#% return - calculate using last value vs first value - similar to pnl calc
percent_return=((Spread_Combi.iloc[-1]-Spread_Combi.iloc[0])/(Spread_Combi.iloc[0]))

In [123]:
pnl_series

1996     247.8
1997     264.6
1998     365.4
1999     277.2
2000     100.8
2001     730.8
2002     289.8
2003     184.8
2004     756.0
2005    1041.6
2006    3628.8
2007   -1684.2
2008    1281.0
2009   -2289.0
2010     924.0
2011    2331.0
2012     659.4
2013    2079.0
2014      71.4
2015     361.2
2016     327.6
2017     718.2
2018     -58.8
dtype: float64

In [124]:
percent_return

1996    0.393333
1997    0.420000
1998    0.378261
1999    0.237410
2000    0.073394
2001    0.535385
2002    0.109524
2003    0.102326
2004    0.456853
2005    0.400000
2006    1.122078
2007   -0.255251
2008    0.241107
2009   -0.396364
2010    0.220662
2011    0.592949
2012    0.119120
2013    0.332661
2014    0.009561
2015    0.036287
2016    0.036094
2017    0.091591
2018   -0.007688
dtype: float64

In [125]:
max_drawdown

1996      -0.0
1997     -67.2
1998     -21.0
1999      -8.4
2000    -499.8
2001    -294.0
2002      -8.4
2003    -575.4
2004      -0.0
2005      -0.0
2006      -0.0
2007   -1684.2
2008      -0.0
2009   -2352.0
2010    -474.6
2011      -0.0
2012    -924.0
2013    -357.0
2014    -121.8
2015   -1243.2
2016    -457.8
2017    -172.2
2018    -121.8
dtype: float64

In [126]:
print 'The winning percentage is %.2f percent' %Win_Percent

The winning percentage is 86.96 percent


In [127]:
#Equity_Curve=create_equity_curve(Spread_Combi, zf[0],zf[1],position)

In [128]:
#Equity_Curve.iplot(kind='line')